In [1]:
m, delta_p, p_min, k, l = 2, 0.5, 0.9, 3, 1000

In [2]:
import pandas as pd
from typing import List
import os

import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 15
})

import pandas as pd
import os

L = 41
DATA = {
        "m":            [m for _ in range(L)],
        "delta_p":      [delta_p for _ in range(L)],
        "p_hat":        [],
        "p_min":        [p_min for _ in range(L)],
        "pi":           [],
        "k":            [k for _ in range(L)],
        "l":            [l for _ in range(L)],
        "search_bound": [],
        "n":            [],
        "i":            []
    }

def read(from_: str) -> pd.DataFrame:
    files = os.listdir(from_)
    for file in files:
        i = float(file.split("_")[0])
        #print(file, i)
        DATA["i"].append(i)
        with open(os.path.join(from_, file), "r") as f:
            #print(f.readlines()[-2:])
            data = next(f.readlines()[-2:].__iter__(), "").rstrip().split(";")
            n, bound, pi, p_hat = float(data[-1]), float(data[-2]), float(data[-5]), float(data[-7])
            DATA["p_hat"].append(p_hat)
            DATA["pi"].append(pi)
            DATA["search_bound"].append(bound)
            DATA["n"].append(n)
    return pd.DataFrame(DATA)

In [3]:
import math

D = "out-fdp-3-50000-600"
#D = "fdp-3-1000-300"
df = read("../out/"+D)

FileNotFoundError: [Errno 2] No such file or directory: '../out/out-fdp-3-50000-600'

In [ ]:
import math
from math import exp

def find_bound_1(bound_2: int, p_hat: float, pi: float) -> int:
    r = (1-p_min) / (m-1)
    for n in range(1,math.ceil(bound_2)+1):
        l = (
            p_hat * exp(-0.5 * n * (delta_p**2) * ((1-pi)**2))
        ) + (
            (1-p_hat) * exp(-0.5 * n * ((delta_p * (1-pi)-pi)**2)) 
        ) 
        v = l <= r
        if v:
            return n
    return 0

df.sort_values(
    by=["p_hat"], 
    inplace=True, 
    ignore_index=True
)
df = df.assign(
    explicit_bound= lambda x: 
        (2 / 
         ((delta_p * (1-x.pi) - x.pi)**2)
        ) * (math.log((m-1) / (1-p_min)))
)
#df["bound_1_min_n"] = df.apply(
df["implicit_bound"] = df.apply(
    lambda x: find_bound_1(x.explicit_bound, x.p_hat, x.pi), axis=1
)
len(df)
df.dropna(inplace=True)
len(df)
df

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(df.p_hat, df.n, 
        "purple", 
        marker="o",
        alpha=0.5,
        label="empirical bound",
    )

ax.plot(df.p_hat, df["explicit_bound"],"purple",alpha=0.5,
        label="explicit bound",linestyle="dotted",marker="o",)
ax.plot(df.p_hat, df["implicit_bound"],"purple",alpha=0.5,
        label="implicit bound",linestyle="dashed",marker="o",)
    
plt.xlim(0, 1)
plt.ylim(1, df["search_bound"].max())
plt.yscale("log")
plt.legend()
plt.xlabel("$\hat{p}$")
plt.ylabel("$n$")
plt.title(
    '$\quad\quad m=2 \quad \Delta p = 0.5 \quad P_{\mathrm{min}} = 0.9$', 
    size=20)
plt.grid(alpha=.5)
plt.tight_layout()
plt.savefig(f"../plots/{D}.pdf")

In [ ]:
df